# Fake News Prediction

## Some important points

1) Filter out the stop words i.e most insignificant word in natural language tool kit.
2) Deal with the null values.
3) Stemming the words to its root word eg- actor, actress to act
4) Save the output result in the new .csv file

Here in this prediction we are using the author name and the title to find it is fake or not

In [40]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.stem.porter import PorterStemmer
import re


# ntlk : nattural language tool kit

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# printing the stopwords in English
# Stop words are the words in a stop list (or stoplist or negative dictionary) which are filtered out (i.e. stopped) before or after processing of natural language data (text) because they are deemed insignificant.
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
df = pd.read_csv('train.csv')

In [5]:
df.head(6)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0


In [6]:
df.shape

(20800, 5)

In [7]:
df.axes

[RangeIndex(start=0, stop=20800, step=1),
 Index(['id', 'title', 'author', 'text', 'label'], dtype='object')]

In [8]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
# Data Pre-processing
# Deal with null values
df.fillna(value ='',inplace = True)
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [10]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [11]:
# Merge the title and author together in the content
df['content'] = df['author']+' ' + df['title']

In [12]:
# And drop the title and author
df.drop(['title','author'], inplace = True,axis=1 )

df

,id,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...
20795,20795,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [13]:
x_col = df.drop("label", axis = 1)
y_col = df['label']
y_col

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

In [14]:
x_col

,id,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...
20795,20795,Rapper T. I. unloaded on black celebrities who...,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,When the Green Bay Packers lost to the Washing...,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [15]:
# Using stemming
# Converting word to the root word to decrease the number of different word.
# Likes,like,liked to like
# actor,actress,act to act


port_stem = PorterStemmer()

In [16]:
def stemming(content):
    # subsitute all the colon,semicolon with a ' ' for the content part only having the a to z or A to Z
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    # Changing to same case
    stemmed_content = stemmed_content.lower()
    # Split where ' ' and return a list
    stemmed_content = stemmed_content.split()
    # using the port_stem.stem to stem the wird which donot include in the stopwords.words
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    # Rejoin the word when the stemming is done
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [17]:
df['content'] = df['content'].apply(stemming)

In [18]:
df

,id,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hous dem aid even see comey letter...
1,1,Ever get the feeling your life circles the rou...,0,daniel j flynn flynn hillari clinton big woman...
2,2,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnew com truth might get fire
3,3,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss civilian kill singl us airstri...
4,4,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy iranian woman jail fiction unpu...
...,...,...,...,...
20795,20795,Rapper T. I. unloaded on black celebrities who...,0,jerom hudson rapper trump poster child white s...
20796,20796,When the Green Bay Packers lost to the Washing...,0,benjamin hoffman n f l playoff schedul matchup...
20797,20797,The Macy’s of today grew from the union of sev...,0,michael j de la merc rachel abram maci said re...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",1,alex ansari nato russia hold parallel exercis ...


In [19]:
# numpy array
x_train = df['content'].values
y_train = df['label'].values

In [20]:
print(x_train)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [21]:
df_test = pd.read_csv('test.csv')

In [22]:
df_test

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...


In [23]:
df_test.fillna(value ='',inplace = True)
df_test.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

In [24]:

# Merge the title and author together in the content
df_test['content'] = df_test['author']+' ' + df_test['title']
df_test.drop('title', axis = 1,inplace = True )
df_test.drop('author', axis = 1,inplace = True )


In [25]:
df_test['content'] = df_test['content'].apply(stemming)

In [26]:
df_test

,id,text,content
0,20800,"PALO ALTO, Calif. — After years of scorning...",david streitfeld specter trump loosen tongu pu...
1,20801,Russian warships ready to strike terrorists ne...,russian warship readi strike terrorist near al...
2,20802,Videos #NoDAPL: Native American Leaders Vow to...,common dream nodapl nativ american leader vow ...
3,20803,"If at first you don’t succeed, try a different...",daniel victor tim tebow attempt anoth comeback...
4,20804,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,truth broadcast network keiser report meme war e
...,...,...,...
5195,25995,Of all the dysfunctions that plague the world’...,jodi rosen bangladeshi traffic jam never end n...
5196,25996,WASHINGTON — Gov. John Kasich of Ohio on Tu...,sheryl gay stolberg john kasich sign one abort...
5197,25997,Good morning. (Want to get California Today by...,mike mcphate california today exactli sushi ne...
5198,25998,« Previous - Next » 300 US Marines To Be Deplo...,us marin deploy russian border norway


In [35]:
x_id = df_test['id']

In [28]:
x_test = df_test['content'].values

In [27]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(x_train)
# Tfid Vectorizer will count the frequency of each word and give a respective numerical value how imp that word is.
# Follows a inverse frequency that the most repeated word has the low importance
x_train = vectorizer.transform(x_train)


In [29]:
x_test = vectorizer.transform(x_test)

In [32]:
lr = LogisticRegression()
lr.fit(x_train, y_train )


LogisticRegression()

In [33]:
y_pred_test = lr.predict(x_test)

In [34]:
y_pred_test

array([0, 1, 1, ..., 0, 1, 0], dtype=int64)

In [38]:
output = pd.DataFrame({ 'NewsId': x_id, 'Label': y_pred_test} )
output

output.to_csv('Logistic_Regression_FakeNews', index = False)

output

,NewsId,Label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1
...,...,...
5195,25995,0
5196,25996,0
5197,25997,0
5198,25998,1
